In [1]:
import pandas as pd
import numpy as np
import mason_functions as mf

# Exercises Part I

In [2]:
#Exercise 3 Create a function named get_db_url. It should accept a username, hostname, password, and database
#name and return a url connection string formatted like in the example at the start of this lesson.
def get_db_url(db_name):
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [3]:
#Exercise 4 Use your function to obtain a connection to the employees database.
sql = """
    SELECT * FROM employees LIMIT 1010
"""

url = get_db_url('employees')
tt = pd.read_sql(sql, url)
tt.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [ ]:
#Exercise 5a  Intentionally make a typo in the database url. What kind of error message do you see?
url = get_db_url('Employees')
q2 = pd.read_sql(sql, url)
#OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for user 'hopper_1550'@'%' to database 'Employees'")
#(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [ ]:
#Exercise 5b Intentionally make an error in your SQL query. What does the error message look like?
sql = """ SELECT * FROM employee"""
url = get_db_url('employees')
q3 = pd.read_sql(sql, url)
#ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'employees.employee' doesn't exist")
#[SQL:  SELECT * FROM employee]
#(Background on this error at: http://sqlalche.me/e/14/f405)
q3

sql = """ 
    SELECT *
    FROM employees
    WHERE first_nam = 'Maya'
"""
q4 = pd.read_sql(sql, url)
q4
#OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'first_nam' in 'where clause'")
#[SQL:  
#    SELECT *
#    FROM employees
#    WHERE first_nam = 'Maya'
#]
#(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [4]:
sql = """ 
    SELECT * FROM titles WHERE to_date != "9999-01-01" ORDER BY to_date DESC LIMIT 1
"""
url = get_db_url('employees')
q4 = pd.read_sql(sql, url)
q4

,emp_no,title,from_date,to_date
0,20278,Senior Staff,1999-11-04,2002-08-01


In [5]:
#Exercise 6 Read the employees and titles tables into two separate DataFrames.
e_sql = """
SELECT *
FROM employees
"""
t_sql = """
SELECT *
FROM titles
"""
employees_table = pd.read_sql(e_sql, url)
titles_table = pd.read_sql(t_sql, url)
e_df = pd.DataFrame(employees_table)
t_df = pd.DataFrame(titles_table)

In [6]:
t_df

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


In [7]:
#Exercise 7 How many rows and columns do you have in each DataFrame? Is that what you expected?
if __name__ == '__main__':
    print(e_df.shape)
    print(t_df.shape)
#kinda

(300024, 6)
(443308, 4)


In [8]:
#Exercise 8 Display the summary statistics for each DataFrame.
t_df.info()
t_df.describe()
e_df.info()
e_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443308 entries, 0 to 443307
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   emp_no     443308 non-null  int64 
 1   title      443308 non-null  object
 2   from_date  443308 non-null  object
 3   to_date    443308 non-null  object
dtypes: int64(1), object(3)
memory usage: 13.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300024 entries, 0 to 300023
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   emp_no      300024 non-null  int64 
 1   birth_date  300024 non-null  object
 2   first_name  300024 non-null  object
 3   last_name   300024 non-null  object
 4   gender      300024 non-null  object
 5   hire_date   300024 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.7+ MB


,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [9]:
#Exercise 9 How many unique titles are in the titles DataFrame?
len(t_df.title.unique())

7

In [10]:
#Exercise 10 What is the oldest date in the to_date column?
oldest_to_date = t_df.sort_values(by = 'to_date').head(1)
oldest_to_date.to_date


16064    1985-03-01
Name: to_date, dtype: object

In [11]:
#Exercise 11 What is the most recent date in the to_date column?
most_recent_to_date = t_df.sort_values(by = 'to_date', ascending = False).head(1)
most_recent_to_date.to_date

0    9999-01-01
Name: to_date, dtype: object

# Exercises Part II

In [12]:
#Exercise 1 Copy the users and roles DataFrames from the examples above.
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users
if __name__ == '__main__':
    print(roles)
    print(users)

   id       name
0   1      admin
1   2     author
2   3   reviewer
3   4  commenter
   id   name  role_id
0   1    bob      1.0
1   2    joe      2.0
2   3  sally      3.0
3   4   adam      3.0
4   5   jane      NaN
5   6   mike      NaN


In [13]:
#Exercise 2 What is the result of using a right join on the DataFrames?
users.merge(roles, left_on = 'role_id', right_on = 'id', how = 'right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


In [14]:
#Exercise 3 What is the result of using an outer join on the DataFrames?
users.merge(roles, left_on = 'role_id', right_on = 'id', how = 'outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


In [ ]:
#Exercise 4 What happens if you drop the foreign keys from the DataFrames and try to merge them?
roles.drop(columns = 'id', inplace = True)
users.drop(columns = 'role_id', inplace = True)
users.merge(roles, left_on = 'role_id', right_on = 'id', how = 'outer')

In [15]:
#Exercise 5 Load the mpg dataset from PyDataset.
from pydataset import data
mpg = data('mpg')
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [16]:
#Exercise 6 Output and read the documentation for the mpg dataset.
data('mpg', show_doc = True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [17]:
#Exercise 7 How many rows and columns are in the dataset?
mpg.shape

(234, 11)

In [18]:
#Exercise 8 Check out your column names and perform any cleanup you may want on them.
mpg.rename(columns = {'cty': 'city'}, inplace = True)
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [19]:
#Exercise 9 Display the summary statistics for the dataset.
mpg.info()
mpg.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   city          234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fl            234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


,displ,year,cyl,city,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


In [20]:
#Exercise 10 How many different manufacturers are there?
len(mpg.manufacturer.unique())

15

In [21]:
#Exercise 11 How many different models are there?
len(mpg.model.unique())

38

In [22]:
#Exercise 12 Create a column named mileage_difference like you did in the DataFrames exercises; this column should
#contain the difference between highway and city mileage for each car.
mpg['mileage_difference'] = mpg.hwy - mpg.city
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,hwy,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


In [23]:
#Exercise 13 Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the
#city and highway mileage.
mpg['average_mileage'] = mpg[['city', 'hwy']].mean(axis = 1)
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,hwy,fl,class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0


In [24]:
#Exercise 14 Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether
#the car has an automatic transmission.
mpg['is_automatic'] = mpg.trans.str.contains('auto')
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,hwy,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False


In [25]:
#Exercise 15 Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?
if __name__ == '__main__':
    print(mpg.groupby('manufacturer').average_mileage.max().nlargest(n = 1, keep = 'all'))
#alternatively
(mpg.groupby('manufacturer').average_mileage.max()).idxmax()

manufacturer
volkswagen    39.5
Name: average_mileage, dtype: float64


'volkswagen'

In [26]:
#Exercise 16 Do automatic or manual cars have better miles per gallon?
mpg.groupby('is_automatic').average_mileage.max()
(mpg.groupby('is_automatic').average_mileage.max()).idxmax()
mpg.groupby('is_automatic').average_mileage.max().nlargest(n = 1, keep = 'all')
#manual cars have better mpg

is_automatic
False    39.5
Name: average_mileage, dtype: float64

# Exercises Part III

In [27]:
#Exercise 1 Use your get_db_url function to help you explore the data from the chipotle database.
url = get_db_url('chipotle')
sql = """
SELECT *
FROM orders
"""
chipotle = pd.read_sql(sql, url)
chipotle.describe()
chipotle.info()
chipotle

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  4622 non-null   int64 
 1   order_id            4622 non-null   int64 
 2   quantity            4622 non-null   int64 
 3   item_name           4622 non-null   object
 4   choice_description  4622 non-null   object
 5   item_price          4622 non-null   object
dtypes: int64(3), object(3)
memory usage: 216.8+ KB


,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75


In [28]:
#Exercise 2 What is the total price for each order?
type(chipotle.groupby('order_id').item_price.sum())
chipotle_orders = chipotle.groupby('order_id').item_price.sum()
chipotle['prices_as_floats'] = chipotle.item_price.apply(lambda x: mf.handle_commas(x))
total_prices = chipotle.groupby('order_id').prices_as_floats.sum()
total_prices

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: prices_as_floats, Length: 1834, dtype: float64

In [29]:
#Exercise 3 What are the most popular 3 items?
chipotle.groupby('item_name').quantity.sum().nlargest(n = 3, keep = 'all')

item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

In [30]:
#Exercise 4 Which item has produced the most revenue?
chipotle.groupby('item_name').prices_as_floats.sum().nlargest(n = 1, keep = 'all')

item_name
Chicken Bowl    7342.73
Name: prices_as_floats, dtype: float64

In [31]:
#Exercise 5 Join the employees and titles DataFrames together.
titles_and_emps = t_df.merge(e_df, on = 'emp_no', how = 'inner')
titles_and_emps.head()

,emp_no,title,from_date,to_date,birth_date,first_name,last_name,gender,hire_date
0,10001,Senior Engineer,1986-06-26,9999-01-01,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,Staff,1996-08-03,9999-01-01,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,Senior Engineer,1995-12-03,9999-01-01,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,Engineer,1986-12-01,1995-12-01,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01,1954-05-01,Chirstian,Koblick,M,1986-12-01


In [32]:
#Exercise 6 For each title, find the hire date of the employee that was hired most recently with that title.
titles_and_emps.hire_date = pd.to_datetime(titles_and_emps.hire_date)
titles_and_emps.from_date = pd.to_datetime(titles_and_emps.from_date)
if __name__ == '__main__':
    print(titles_and_emps.groupby('title').from_date.max())
    print(titles_and_emps.groupby('title').hire_date.max())

title
Assistant Engineer   2000-02-01
Engineer             2002-08-01
Manager              1996-08-30
Senior Engineer      2002-08-01
Senior Staff         2002-08-01
Staff                2000-02-01
Technique Leader     2000-02-01
Name: from_date, dtype: datetime64[ns]
title
Assistant Engineer   1999-12-24
Engineer             2000-01-28
Manager              1992-02-05
Senior Engineer      2000-01-01
Senior Staff         2000-01-13
Staff                2000-01-12
Technique Leader     1999-12-31
Name: hire_date, dtype: datetime64[ns]


In [33]:
#Exercise 7 Write the code necessary to create a cross tabulation of the number of titles by department. 
#(Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform 
#the manipulations.)
url = get_db_url('employees')
sql = """
SELECT title, dept_name
FROM employees
JOIN titles USING(emp_no)
JOIN dept_emp USING(emp_no)
JOIN departments USING(dept_no)
WHERE dept_emp.to_date > NOW()
"""
the_crosst = pd.read_sql(sql, url)
pd.crosstab(the_crosst.title, the_crosst.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,240,5551,0,0,0,4670,1280,316,0
Engineer,1927,42111,0,0,0,36308,10174,2416,0
Manager,4,2,2,2,2,4,4,2,2
Senior Engineer,1790,38816,0,0,0,33627,9459,2250,0
Senior Staff,11271,1085,9546,9825,11291,1123,0,9093,28797
Staff,11836,1123,9995,10407,11885,1185,0,9646,30117
Technique Leader,241,5478,0,0,0,4724,1295,321,0
